In [1]:
import datasets
datasets.logging.set_verbosity_error()

In [2]:
from matplotlib import rcParams
rcParams.update({'xtick.labelsize': 14, 'ytick.labelsize': 14, 'axes.labelsize': 16})

import numpy as np

In [3]:
# import argparse


# parser = argparse.ArgumentParser()
# parser.add_argument('--top_k', type=int, default=1,
#                     help='Sample from top k predictions,you canot change this parameter when it is used for binary classification')
# parser.add_argument('--beta1', type=float, default=0.9,
#                     help='hyperpara-Adam')
# parser.add_argument('--eval_iter', type=int, default=100,
#                     help='Sample generator output evry x steps')
# parser.add_argument('--save_para_every', type=int, default=100,
#                     help='save model parameters every')

# parser.add_argument('--datapath', type=str, default='Data/Session/LFDshort1w.csv',
#                     help='data path')
# parser.add_argument('--tt_percentage', type=float, default=0.9,
#                     help='default=0.2 means 80% training 20% testing')
# parser.add_argument('--is_generatesubsession', type=bool, default=False,
#                     help='whether generating a subsessions, e.g., 12345-->01234,00123,00012  It may be useful for very some very long sequences')
# parser.add_argument('--has_positionalembedding', type=bool, default=False,
#                     help='whether contains positional embedding before performing cnnn')

# parser.add_argument('--padtoken', type=str, default='0',
#                     help='is the padding token in the beggining of the sequence')
# parser.add_argument('--negtive_samples', type=int, default='5',
#                     help='the number of target class -1, e.g. for binary classification it is 1 ,if the class is 7, then it is 6')
# parser.add_argument('--is_shuffle', type=bool, default=False,
#                     help='whether shuffle the training and testing dataset, e.g., 012345-->051324')
# args = parser.parse_args()
import data_loader_recsys_transfer_finetune_ as data_loader_recsys


dl = data_loader_recsys.Data_Loader({'model_type': 'generator', 'dir_name': 'Data/Session/LFDshort1w.csv'})

all_samples = dl.example
print(("len(all_samples)", len(all_samples)))
items = dl.item_dict
items_len = len(items)
print(("len(items)", len(items)))
targets = dl.target_dict
targets_len=len(targets)
print(("len(targets)", len(targets)))

negtive_samples=5
top_k=1

if 0 in items:
    padtoken = items[0]  # is the padding token in the beggining of the sentence
else:
    padtoken=len(items)+1

np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(all_samples)))
all_samples = all_samples[shuffle_indices]

dev_sample_index = -1 * int(0.9 * float(len(all_samples)))
train_set, valid_set = all_samples[:dev_sample_index], all_samples[dev_sample_index:]

C:\tools\Miniconda3\envs\ai6103-project\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\tools\Miniconda3\envs\ai6103-project\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\tools\Miniconda3\envs\ai6103-project\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\tools\Miniconda3\envs\ai6103-project\lib\site-packages\ten

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
('len(all_samples)', 10000)
('len(items)', 117762)
('len(targets)', 7)


In [4]:
type(train_set[19])

numpy.ndarray

In [5]:
from small_text.data.datasets import SklearnDataset

train_set_active = SklearnDataset(train_set[:,0:-1], train_set[:,-1], [1,2,3,4,5,6])

In [6]:
type(train_set_active.x)

numpy.ndarray

In [7]:
from small_text.active_learner import PoolBasedActiveLearner
from small_text.initialization import random_initialization_balanced
from small_text.query_strategies import PredictionEntropy
from small_text.classifiers import ConfidenceEnhancedLinearSVC
from small_text.classifiers.factories import SklearnClassifierFactory



model = ConfidenceEnhancedLinearSVC()
num_classes = 6
clf_factory = SklearnClassifierFactory(model, num_classes)
query_strategy = PredictionEntropy() # changeable
active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, train_set_active)

In [8]:
def initialize_active_learner(active_learner, y_train):
    indices_initial = random_initialization_balanced(y_train, n_samples=20)
    active_learner.initialize_data(indices_initial, y_train[indices_initial])

    return indices_initial

indices_labeled = initialize_active_learner(active_learner, train_set_active.y)


In [9]:
train_set_active.x[0]

array([    0,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
        5021, 26184,  2152,  3461,  2178,  1189, 26185,  2148,  3215,
       26186,  3042,     0], dtype=int64)

In [10]:
def generate_train_data(indices_labeled, train_set_active):
    to_return = np.array([],dtype=np.int64)
    new_return = train_set_active.x[indices_labeled]
    new_y = train_set_active.y[indices_labeled].reshape((20,1))
#     print(new_return.dtype)
#     print(new_y.shape)
    for i in range(0, 20):
        to_return = np.append(to_return, new_return[i])
        to_return = np.append(to_return, new_y[i])
#     print(to_return.shape)
    return to_return.reshape((20,-1))

In [11]:
to_train = generate_train_data(indices_labeled, train_set_active)

In [19]:
type(indices_labeled)

numpy.ndarray

In [ ]:
for i in range(20):
    indices_queried = active_learner.query(num_samples=20)
    
    to_train = np.append(to_train, generate_train_data(indices_queried, train_set_active))
    # Simulate user interaction here. Replace this for real-world usage.
    y = train_set_active.y[indices_queried]

    # Return the labels for the current query to the active learner.
    active_learner.update(y)
    indices_labeled = np.concatenate([indices_queried, indices_labeled])

In [ ]:
for i in range(20):
    indices_queried = active_learner.query(num_samples=20)
    
    to_train = np.append(to_train, generate_train_data(indices_queried, train_set_active))
    # Simulate user interaction here. Replace this for real-world usage.
    y = train_set_active.y[indices_queried]

    # Return the labels for the current query to the active learner.
    active_learner.update(y)
    indices_labeled = np.concatenate([indices_queried, indices_labeled])

In [ ]:
to_train.reshape((-1,103)).shape

In [21]:
import tensorflow as tf
tf.test.is_gpu_available()

False